# Alignement de signaux

In [ ]:
%reset -f
import numpy as np
import scipy
from scipy import signal as sg
from scipy import ndimage
import  matplotlib.pyplot as plt

np.set_printoptions(precision=5,linewidth=5000,suppress=True)

## Convolution par FFT


### Fourier discret


On note $\mathcal F_N$ la transformation de Fourier discrète de taille $N$: Pour $u=(u_0,...,u_{N-1})$:
$$
\forall k\in\{0,...,N-1\}\qquad  \mathcal F_N[u]_k = \frac 1 N \sum_{j=0}^{N-1} u_j e^{- 2i\pi jk /N}
$$
C'est une bijection de $\mathbb C^N$ dans lui-même. Son inverse est:
$$
\mathcal F^{-1}_N[u]_k = \sum_{j=0}^{N-1} u_j e^{+ 2i\pi jk/N}
$$

Remarque: si l'on décide de ne pas multiplier par $\frac 1 N$ dans la définition de la transformée de Fourier, on obtient des coefficients de Fourier $N$ fois trop grands. Il faut donc multiplier par $\frac 1 N$ dans la formule de l'inverse.





###  Convolution circulaire

Soient $u,v\in \mathbb C^N$.  On définit la convolution circulaire de taille $N$ par:
$$
\forall k\in \{0,...,N-1\} \qquad  u \stackrel N \ast v (k) = \sum_{t=0}^{N-1} u_t v_{k \dot - t }
$$
où $\dot -$ est la soustraction modulo $N$


Exemple: Soit $m =[\frac 1 3, \frac 1 3, 0, 0, ...., \frac 1 3]$. Si on pense modulo $N$, ce vecteur peut être vu comme un créneau centré autour de 0.   Si $u$ est un signal, alors  $ u \stackrel N \ast m $ est le siganl $i\to \frac 1 3 [u_{i-1}+u_i + u_{i+1}]$ avec $i-1$ et $i+1$ pris modulo $N$. Cette convolution effectue donc une moyenne mobile circulaire.   


---
**Théorème:** On a:
 $$
  \mathcal F_N [u \stackrel N \ast v] =  N \ \mathcal F_N [u] \mathcal F_N [v]
 $$

 ---


Preuve: il suffit d'effectuer un changement de variable. Nous écrivons le début et laisson la fin au lecteur:   
 \begin{alignat}{1}
    \mathcal F_N [u \stackrel N \ast v](j)&= \frac 1 N \sum_{k=0}^{N-1}  e^{-2i\pi \frac {kj}N  }   \Big(   \sum_{t=0}^{N-1} u(t) v[(k \dot -t)  ]    \Big)
 \end{alignat}
On effectue le changement de variable $(k \dot -t) \to t' $...



***A vous:*** En quoi les modulos sont-ils important pour que la prevue fonctionnent ?


 On ne peut faire des changements de variable qu'avec des bijection. Or sans le module, la fonction $k-t -\to t'$ n'est pas une bijection.


---
**Corollaire 1:** On a:
  $$
  \mathcal F^{-1}_N [u \stackrel N \ast v] =   \mathcal F^{-1}_N [u] \mathcal F^{-1}_N [v]
  $$

---


Preuve: On part du théorème:
$$
  \mathcal F_N [u \stackrel N \ast v] =  N \ \mathcal F_N [u] \mathcal F_N [v]
 $$
 Que l'on applique avec $u:=\bar u$ et $v:=\bar v$
 $$
  \mathcal F_N [\bar u \stackrel N \ast \bar v] =  N \ \mathcal F_N [\bar u] \mathcal F_N [\bar v]
 $$
 On ajoute une barre au dessus et multiplie par N:
$$
    \mathcal F^{-1}_N [ u \stackrel N \ast  v] =   \ \mathcal F^{-1}_N [ u]   \mathcal F^{-1}_N [ v]
 $$

où l'on a utilisé la relation:
$$
 \mathcal F_N^{-1}[w] =   N    \overline  {\mathcal F_N[\bar w ]}
$$




---
**Corollaire 2:** Notons pas $\times$ la multiplication terme à terme de deux vecteurs: $(u \times  v)_k = u_k v_k$. On a:
 $$
 \mathcal F_N [ u \times     v] =    \mathcal F_N [u] \stackrel N \ast  \mathcal F_N [v]
 $$

 ---


Preuve:
\begin{alignat*}{1}
 \mathcal F_N [u] \stackrel N \ast  \mathcal F_N [v] &=   \mathcal F_N  \Big[\mathcal F_N^{-1} \Big(  \mathcal F_N [u] \stackrel N \ast  \mathcal F_N [v]  \Big) \Big]   \\
&= \mathcal F_N  \Big[ \mathcal F_N^{-1} (  \mathcal F_N [u] )   \times    \mathcal F_N^{-1}   (\mathcal F_N [v]  ) \Big]   \\
  &= \mathcal F_N(u \times v )
\end{alignat*}


###  Filtering by FFT or by convolution.

During the T.P about 1D signal, we did filtering as follows:


We take a discrete signal  $u$, we choose  $\hat m$ a vector that cancels out over certain index ranges (i.e. a frequency mask, cancelling out over a range of frequency ). Then we  construct a new signal:
 \begin{alignat}{1}
    u'=  \mathcal F_N^{-1} \Big[  \mathcal F_N[u]  \times \hat m \Big]
 \end{alignat}

Example: if $\hat m$ is zero on the hight frequency, the signal $u'$ is constituated by the low frequency of $u$.

But we can always imagine that the frequency mask $\hat m$ is derived from Fourier transform of a time mask $m$ (i.e. $\hat m = \mathcal F_N[m]$) and then we can rewrite the previous equation by
$$
 u'=  \mathcal F_N^{-1} \Big[  \mathcal F_N[u] \mathcal F_N[m] \Big] = u  \stackrel N \ast m
 $$
 This convolution filtering is what we used for the images.

Note: FFT requires $cst N \log N$ operations while convolution $cst N ^2$ operations. FFT filtering is faster ...    except when the $m$ support has a small $r$ size, in which case the convolution takes $r N$ operations.




### Fast convolution


We have just seen that the FFT can calculate circular convolutions quickly. But usualy we want to perform non-circular convolutions of $u,v$:
$$
u \ast v (k) = \sum_{t\leq k}  u_t v_{k-t}
$$
( the condition $t\leq k$ can also be replaced by the convention of saying that vectors cancels on negative indices).

To perform this operation via a circular convolution, we use a trick:  suppose that   $u$ and $v$ have size less than $N$.  We considere them as element of $\mathbb C^{2N}$ by extending them with zeros.

Ex: $N=4$, $u=[1, 2, 3,4]$ , $ v=[-1,+1]$ then we considere them as $[1, 2, 3,4,0,0,0,0]$ and $[-1,+1,0,0,0,0,0,0]$.

Then we remark that:
$$
 (u \ast v)_{i}    =  ( u \stackrel {2N} \ast v)_{i}
$$
Indeed: the side effect created by the modulo $2N$ make no interferences because of all the additionnals 0.




So we can use $\mathcal F_{2N}$ to compute the true convolution with the help of $ cst (2N) \log (2N)$ operations.


Exercise: show that multiplying two polynomials implies a convolution (on the coefficients of the two polynomials).

Exercise: show that multiplying two integers implies a convolution (on the digits of the two integers).

This give us some fast algorithm to multiply polynomials or integers.




## Practice  1D



### Classical convolution  

In [ ]:
signal1=[1,2,3,4]
signal2=[1,0,2]

In [ ]:
np.convolve(signal1,signal2)

In [ ]:
scipy.signal.convolve(signal1,signal2)

The formula is:

    convo[x] = sum_i   signal1[i] signal2[x-i]
    for x>=0, and the sum is taken on 0<=i<=x


To find the result `[1, 2, 5, 8, 6, 8]` graphicaly:

* we return `signal2`
* decay it by x
* multiply aligned term
* sum up.


```
        [1,2,3,4]
                  x=0 ---> 1
    [2,0,1]


        [1,2,3,4]
                  x=1 ---> 2
      [2,0,1]


        [1,2,3,4]
                 x=2 ---> 5
        [2,0,1]


        [1,2,3,4]
                 x=3 ---> 8
          [2,0,1]


        [1,2,3,4]
                 x=4 ---> 6
            [2,0,1]


        [1,2,3,4]
                 x=5 ---> 8
               [2,0,1]

```

Now we use FFT to compute this same convolution. To avoid the cyclicity, we prolongate initial signal by many zeros:

In [ ]:
def fast_convolution(signal1, signal2):
    max_len = max(len(signal1), len(signal2))

    """we double the size, to avoid the 'circular' effect of the convolution via fft """
    enlarged1 = np.zeros(2*max_len)
    enlarged2 = np.zeros(2*max_len)
    enlarged1[:len(signal1)] = signal1
    enlarged2[:len(signal2)] = signal2

    return np.fft.irfft(np.fft.rfft(enlarged1)*np.fft.rfft(enlarged2))

***To you:*** Modify our function `fast_convolution` in order to have exactly the same result as `np.convolution`; so you have to suppress the additionnal zeros at the end. The simplest way is to add less zeros to your vectors. So change `max_len =...`

In [ ]:
#@title Correction
def fast_convolution(signal1, signal2):
    final_length = len(signal1) + len(signal2) - 1

    """we double the size, to avoid the 'circular' effect of the convolution via fft """
    enlarged1 = np.zeros(final_length)
    enlarged2 = np.zeros(final_length)
    enlarged1[:len(signal1)] = signal1
    enlarged2[:len(signal2)] = signal2

    #Notez qu'il faut indiquer à la fonction irfft la taille des vecteurs, car cette fonction ne peut pas la déduire à partir de sa seule entrée:
    # size=n --rfft-->  size=n//2 + 1:=m  -- irfft -> ??? 2m ou bien 2m-1
    #.     3                 2
    #      4                 3
    return np.fft.irfft(np.fft.rfft(enlarged1)*np.fft.rfft(enlarged2),n=final_length)

signal1=np.random.uniform(0,1,size=6)
signal2=np.random.uniform(0,1,size=5)
print(fast_convolution(signal1,signal2))
print(np.convolve(signal1,signal2))

### Modes 'full', 'valid', 'same'

In [ ]:
print(signal1)
print(signal2)

What we did before is:

In [ ]:
np.convolve(signal1,signal2,mode='full') #full is the default option

In [ ]:
np.convolve(signal1,signal2,mode='valid')

With the 'valid' mode, the second input is slided only inside the first one.

```
        [1,2,3,4]
                  ---> 5
        [2,0,1]


        [1,2,3,4]
                 ---> 8
          [2,0,1]
```

In [ ]:
np.convolve(signal1,signal2,mode='same')

In the 'same' mode, the output have the same length as the first input.

```    
        [1,2,3,4]
                  x=1 ---> 2
      [2,0,1]


        [1,2,3,4]
                 x=2 ---> 5
        [2,0,1]


        [1,2,3,4]
                 x=3 ---> 8
          [2,0,1]


        [1,2,3,4]
                 x=4 ---> 6
            [2,0,1]
```

***To you:*** For which modes the convolution is commutative?

### Performance

FFT :
pour calculer la transformée 'd'un vecteur de taille N
on calcule 2 transformée de vecteur de taille N/2



In [ ]:
import numpy as np

In [ ]:
long1=np.random.uniform(0,1,size=100_000)
long2=np.random.uniform(0,1,size=100_000)

In [ ]:
%%time
np.convolve(long1,long2)

In [ ]:
%%time
fast_convolution(long1,long2)

So we do better than numpy. Waou! But `scipy` can do the same:

In [ ]:
import scipy.signal
scipy.signal.convolve?

The function convolve as an argument
`method` which an be:
* 'auto' (by default)
* 'direct'
* 'fft'

***To you:*** What is the meaning of these keywords.

### Circular convolution

This operation is less useful, but we can check it anyway.

This convolution is theoriticaly defined for two signals of the same size N.

But for practicity, we take N as the size of the longest signal, and extend the shortest with 0 to get the same size.

The formula is:

            x-> sum_i  signal1[i] signal2[x-i].

where

        x,i are in {0,...,N-1}
        the substraction x-i is modulo N.

In [ ]:
def fast_circular_convolution(signal1,signal2):
    N = max(len(signal1), len(signal2))

    enlarged1 = np.zeros(N)
    enlarged2 = np.zeros(N)
    enlarged1[:len(signal1)] = signal1
    enlarged2[:len(signal2)] = signal2
    return np.real(np.fft.ifft(np.fft.fft(enlarged1)*np.fft.fft(enlarged2)))

In [ ]:
fast_circular_convolution(signal1,signal2)

```
        [1,2,3,4]
              ---> 7
         1] [2,0


        [1,2,3,4]
              ---> 10
         0,1] [2,


        [1,2,3,4]
              ---> 5
        [2,0,1]


        [1,2,3,4]
              ---> 8
          [2,0,1]

```

##  Signal matching

As  explained in the theoretical part, the convolution can be use for signal filtering.
* With wide mask we use fast convolution (or directly FFT filtering)
* With small mask we use classical convolution

Usualy, for 2D signal as images, the mask are small. While for 1D signal as sound, the mask can be wide.


But convolution is frequently used for another purpose: signal matching: we want to find the best translation to overlay two signals.    

### Simple picks

In [ ]:
"""creation of 2 single hump signals"""
def twoSignals(N):

    x=np.arange(0,N)
    taille=int(N/20)
    bosse=np.concatenate([np.arange(0,taille)/taille,-np.arange(0,taille)/taille+1])

    signal1=np.zeros(N)
    signal2=np.zeros(N)

    signal1[int(N*0.6):int(N*0.6)+2*taille]=bosse
    signal2[int(N*0.3):int(N*0.3)+2*taille]=np.sqrt(bosse)

    return signal1,signal2

In [ ]:
"""let's look at the two signals"""
N = 100
signal1,signal2=twoSignals(N)
plt.plot(signal1,label="signal1")
plt.plot(signal2,label="signal2")
plt.legend();

In [ ]:
convo=np.convolve(signal1,signal2[::-1])#mode="full"
best_decay=np.argmax(convo)-len(signal2)+1
best_decay

In [ ]:
# just inverse the two signal:
signal2,signal1=twoSignals(N)
convo=np.convolve(signal1,signal2[::-1])
best_decay=np.argmax(convo)-len(signal2)+1
best_decay

The general formula is:


    best_decay of the signal 2 =  np.argmax(convo) - len(signal2) + 1


case 1: negative best decay

                signal1
            ---/\-------------

          signal2
    -----------/\-----

                x=np.argmax(convo)
            --------->



case 2: positive best decay

                signal1
            ------------/\----

                           signal2
                     ---/\-------------

                         x=np.argmax(convo)
            -------------------------->



In [ ]:
"""let's come back to the first case:"""
signal1,signal2=twoSignals(N)

convo=np.convolve(signal1,signal2[::-1])
best_decay=np.argmax(convo)-len(signal2)+1
print("best_decay:",best_decay)

signal2_decay=np.concatenate([np.zeros(best_decay),signal2])
signal1_decay=np.concatenate([signal1,np.zeros(best_decay)])
#np.pad peut aussi servir

plt.plot(signal1_decay,label="signal1_dec")
plt.plot(signal2_decay,label="signal2_dec")
plt.legend();

***To you:***
* Make the translation in the second case where the best decay is negative.
* Find the matching when the two signals have not the same size. Very few think to change. Example of two such signals below.
* Make a general programme that allows to mathch two signals.

In [ ]:
signal1,signal2=twoSignals(N)
signal2=signal2[:60]

fig,(ax0,ax1)=plt.subplots(2,1,sharex=True)
ax0.plot(signal1)
ax1.plot(signal2);

### Signals  that do not cancel out at the edge

Now we add 10 to both signals. And that creates problems!

In [ ]:
N = 100
signal1, signal2 = twoSignals(N)
signal1+=10
signal2+=10

fig,(ax0,ax1)=plt.subplots(2,1,sharex=True)
ax0.plot(signal1)
ax1.plot(signal2);

In [ ]:
convo=np.convolve(signal1,signal2[::-1])
best_decay=np.argmax(convo)-len(signal2)+1
best_decay

***To you:*** Explain the problem. Find a solution. Modify your previous signal matching program to implement this solution.

If you have no idea, the solution is perhaps hidden latter on.
    

## La meilleurs translation à coefficient multiplicatif et additif près.



Dans cette section nous considérons des vecteurs de taille $N$, les indices étant considérés modulo $N$ (sela modèlise des signaux périodiques).


On a un signal $u$ issu d'observation. On a un second signal $v$ qui est issu d'un modèle. On aimerait caller au mieux le modèle $v$ sur l'observation $u$, mais en s'autorisant toutes les translations, les additions de constantes et les multiplication par des constantes. En notant $v_t$ le signal translaté de $t$ (donc $v_{t,i} = v_{i-t}$),  on cherche donc à résoudre le problème de minisation suivant:
$$
\min_{t,a,b}\|  u - a v_t + b\|^2  
$$
où $\|  u - a v_t + b\|^2 = \sum_i (u_i -a v_{i-t} - b)^2 $



On cherche non seulement la valeur de ce minium, mais aussi son lieu:
$$
\underset{t,a,b}{\text{argmin}} \|  u - a v_t + b\|^2
$$



> ***Lemme:*** Notons $\dot u$ et $\dot v$ les versions centrées des vecteurs $u$ et $v$, alors on a:
$$
\min_{t,a,b}\|  u - a v_t + b\|^2=\min_{t,a,b}\|  \dot u - a \dot v_t + b\|^2    
$$
et si
$$
\underset{t,a,b}{\text{argmin}} \|  u - a v_t + b\|^2 = \hat t, \hat a, \hat b    
$$
alors
$$
\underset{t,a,b}{\text{argmin}} \|  \dot u - a \dot v_t + b\|^2 =
\hat t, \hat a,  \hat b \pm (\text{mean}(u)-\hat a \, \text{mean}(v))    
$$


Preuve: C'est à vous de faire: écrivez: $\|  u - a v_t + b\|=\|  \dot u - a \dot v_t + b + cst \|$ puite faite le changement de variable qui va bien, et décidez si le $\pm$ est un $+$ ou bien un $-$.  

A partir de maintenant nous remplaçons les vecteurs $u$ et $v$ initiaux par leur vecsions centrées, donc
$$
\sum_i u_i = 0 \text{ et } \sum_i v_i = 0
$$
La solution du problème initiale se déduira facilement de la solution du problème "centré" au moyen du lemme précédent.



Par ailleurs nous notons:
$$
K= \sum_i v_i^2
$$


Considérons déjà le problèmes de minimisation pour un $t$ fixé:
$$
\underset{t,a,b}{\text{argmin}} \|  u - a v_t + b\|^2  
$$
On reconnait ici le problème d'ajustement affine classique qui se résoud par "l'équation normale". Détaillons: en notans $V_t$ la matrice dont la première colonne est constituée de 1 et la seconde du vecteur $v_t$, le minimum est atteint pour:
$$
\begin{pmatrix} \hat b_t \\ \hat a_t \end{pmatrix}= (V^T_tV_t)^{-1}V_t^T U
$$
où $U$ c'est le vecteur $u$ disposé en colonne. Comme on a pris $u$ et $v$ centrés, le calcul est très simple:
$$
\begin{pmatrix} \hat b_t \\ \hat a_t \end{pmatrix}= \begin{pmatrix} 0 \\
 \frac{1}{K} \sum_i u_i v_{i-t} \end{pmatrix}
$$
Notons que $\hat a_t$ s'exprime en terme de convolution:   
$$
\hat a_t = \frac {(u\ast \check v )(t)} K
$$
où $\check v$ est le vecteur retourné.


Reprenons le problème complet: La meilleure translation est donc:
$$
\hat t= \underset{t}{\text{argmin}}\Big[ \|  u - \hat a_t v_t\|^2  \Big]
$$
On développe les carrés:
$$
\hat t= \underset{t}{\text{argmin}}\Big[ \sum_i  (u_i)^2  + (\hat a_t)^2 (v_{i-t})^2 - 2 u_i \hat a_t v_{i-t} \Big]
$$
$$
= \underset{t}{\text{argmin}} \Big[(\hat a_t)^2  \sum_i (v_{i-t})^2 - 2 \hat a_t \sum_i u_i v_{i-t}  \Big]
$$
$$
= \underset{t}{\text{argmin}} \Big[(\hat a_t)^2 K - 2 \hat a_t K \hat a_t\Big]
$$
$$
= \underset{t}{\text{argmax}}\  (\hat a_t)^2 = \underset{t}{\text{argmax}}\  (u\ast \check v )(t)
$$

Résumons tout cela en un théorème:



> ***Théorème:*** si $u$ et $v$ sont centrés, alors
$$
\underset{t,a,b}{\text{argmin}}\|  u - a v_t + b\|^2  = \hat t, \hat a_{\hat t} , 0
$$
où $\hat t$ et $\hat a_{\hat t}$ sont le lieu et la valeur du maximum de $$
t\to \frac {(u \ast \check v)(t)}{\sum_i v_i^2}
$$



***A vous:*** Repérez tous les endroits dans la preuve ou on a utiliser le fait que les indices étaient périodiques.




## Image matching

Now we work in 2D

### Load images

In [ ]:
"we import (=clone) all the data or just update (=pull) them"

import os

if not os.path.exists("assets_signal"):
    print("the directory assets_signal is create")
    !git clone https://github.com/vincentvigon/assets_signal
else:
    print("the directory assets_signal is updated")
    %cd assets_signal
    !git pull https://github.com/vincentvigon/assets_signal
    %cd ..

In [ ]:
%ls

In [ ]:
def get_simple_images(choix):
    path="assets_signal/imageMatching/"

    img1=None
    img2=None
    if choix==0:
        img1=  plt.imread(path+"gamma_fin1.gif")[:,:,0]
        img2 = plt.imread(path+"gamma_fin1.gif")[:,:,0]
    elif choix == 1:
        img1 = plt.imread(path+"gamma_grand.gif")[:, :, 0]
        img2 = plt.imread(path+"gamma_petit.gif")[:, :, 0]
    elif choix==2:
        img1 = plt.imread(path+"grand_triangle.gif")[:, :, 0]
        img2 = plt.imread(path+"petit_triangle.gif")[:, :, 0]
    elif choix==3:
        img1 = np.ones((3,3))
        img2= np.ones((2,2))

    elif choix==4:
        img1 = plt.imread(path+"king.gif")[:, :, 0]
        img2 = plt.imread(path+"translated.gif")[:, :, 0]
        """que fait-t-on subir à nos images ci-dessous ? """
        img1 = 255 - img1  # type:np.ndarray
        img2 = 255 - img2  # type:np.ndarray

    elif choix == 5: #idem qu'avant, mais en inversant l'ordre des images
        img1 = plt.imread(path+"translated.gif")[:, :, 0]
        img2 = plt.imread(path+"king.gif")[:, :, 0]
        """que fait-t-on subir à nos images ci-dessous ? """
        img1 = 255 - img1  # type:np.ndarray
        img2 = 255 - img2  # type:np.ndarray


    elif choix == 6:
        img1 = plt.imread(path+"lena/lena_256_NB.png")
        img2 = plt.imread(path+"lena/translation1.png")[:,:,0]

        print(img1.shape)
        print(img2.shape)


    elif choix == 7:
        img1 = plt.imread(path+"lena/translation1.png")
        img2 = plt.imread(path+"lena/translation1.png")[:, :, 0]

        print(img1.shape)
        print(img2.shape)


    elif choix == 10:
        img1 = plt.imread(path+"lena/lena_256_NB.png")
        img2 = plt.imread(path+"lena/translation1.png")[:,:,0]


    elif choix == 11:
        img1 = plt.imread(path+"lena/translation1.png")
        img2 = plt.imread(path+"lena/translation2.png")[:, :, 0]



    return img1.astype(np.float32),img2.astype(np.float32)


### Observe convolutions

In [ ]:
import scipy.signal
def observe(choice):
    img1,img2=get_simple_images(choice)
    fig,(ax0,ax1,ax2)=plt.subplots(1,3,figsize=(15,5))
    ax0.matshow(img1)
    ax1.matshow(img2);
    #Do not use `scipy.signal.convolve2d` which is a slow convolution.
    convo=scipy.signal.convolve(img1,img2[::-1,::-1])
    ax2.imshow(convo)

In [ ]:
observe(1)

In [ ]:
observe(2)

In [ ]:
# convolution of 2 constants (non zero) images
observe(3)

In [ ]:
size=10
img1=np.ones([size,size])
img2=np.ones([size,size])
convo=scipy.signal.convolve(img1,img2[::-1,::-1])
plt.matshow(convo)

In [ ]:
observe(4)

### Shift an image

To match images, we need to be able to decay them. Here are some utily function. Be free to modify them as you want.

In [ ]:
def wrapping_shape(img1:np.ndarray, img2:np.ndarray):
    if len(img1.shape) != 2 or len(img2.shape) != 2: raise ValueError("args must be matrix")
    shape1 = np.array(img1.shape)
    shape2 = np.array(img2.shape)
    return np.maximum(shape1, shape2)


def enlargeImage(img:np.ndarray,enlargedShape)->np.ndarray:
    if len(img.shape)!=2 : raise ValueError("args must be matrix")
    if img.shape[0]>enlargedShape[0] or img.shape[1]>enlargedShape[1]: raise ValueError("enlargedShape must be larger that the shape of the make_image to enlarge")
    res=np.zeros(enlargedShape)
    res[:img.shape[0],:img.shape[1]]=img
    return res


def shiftAndEnlargeImage(img:np.ndarray, shift, enlargedShape=None)->np.ndarray:
    if enlargedShape is None : enlargedShape=(img.shape[0]+shift[0],img.shape[1]+shift[1])
    res=np.zeros(enlargedShape)
    res[shift[0]:img.shape[0]+shift[0],shift[1]:img.shape[1]+shift[1]]=img
    return res


In [ ]:
#test
img1,img2=get_simple_images(1)
wrap=wrapping_shape(img1,img2)

plt.subplot(2,2,1)
plt.imshow(enlargeImage(img1, wrap),vmin=0, vmax=255)
plt.subplot(2,2,2)
plt.imshow(enlargeImage(img1,wrap * 2))
plt.subplot(2, 2, 3)
plt.imshow(shiftAndEnlargeImage(img1, (20, 20)))
plt.subplot(2, 2, 4)
plt.imshow(shiftAndEnlargeImage(img1, (30, 10), enlargedShape=wrap * 2));

### argmax in 2D

the function `np.argmax` give the argmax of the flattened version of a matrix. So we need to use the function `bp.unravel_index` to find the good couple of index.  

In [ ]:
a=np.zeros(shape=(5,5))
a[3,2]=1
argMax=np.argmax(a)
print("argMax",argMax)
print("unravel_index",np.unravel_index(argMax,(5,5)))

### King matching


***To you:*** Try to supperpose the two following kings.

In [ ]:
observe(4)

***To you:*** It could be nice if your programme also work with in this case:

In [ ]:
observe(5)

It could be nice if you program works in any situations. Their are 4 types of possible decay:

* x-decay>0, y-decay>0

* x-decay>0, y-decay<0

* x-decay<0, y-decay>0

* x-decay<0, y-decay<0


###  Lena matching

In [ ]:
observe(10)

Because the second image is "include" in the second one, which mode can be used in the convolution ("full", "valid", "same") ?

Your first try will probably fail: the front of Lena match prety well with the hat of Lena :-)

 Try again after remembering of what we did in 1D.

## Transformations

### Load images

In [ ]:
def get_turned_images(choix, centerReduce=True):

    img1=None
    img2=None

    path="assets_signal/imageMatching/"

    if choix == 0:
        img1 = plt.imread(path+"king_head_turn.gif")[:, :, 0]
        img2 = plt.imread(path+"king.gif")[:, :, 0]
        """que fait-t-on subir à nos images ci-dessous ? """
        img1 = 255 - img1  # type:np.ndarray
        img2 = 255 - img2  # type:np.ndarray


    elif choix == 1:
        img1 = plt.imread(path+"king_head_zoom_turn.gif")[:, :, 0]
        img2 = plt.imread(path+"king.gif")[:, :, 0]

        img1 = 255 - img1  # type:np.ndarray
        img2 = 255 - img2  # type:np.ndarray




    img1,img2=img1.astype(np.float32), img2.astype(np.float32)
    if centerReduce:
        img1-=img1.mean()
        img1/=img1.std()
        img2 -= img2.mean()
        img2 /= img2.std()



    return img1,img2

In [ ]:
img1,img2=get_turned_images(0)

fig,(ax0,ax1)=plt.subplots(1,2)
ax0.imshow(img1)
ax1.imshow(img2);

### Find the best rotation

In [ ]:
"""we try all the rotation and look for the one which give the best matching  """

values=[]
angles=np.arange(0,360,10)
for angle in angles:
    rot = ndimage.rotate(img1, angle, order=2)
    cor = scipy.signal.convolve(rot, img2[::-1,::-1])
    values.append(np.max(cor))

plt.plot(angles,values);

***To you:*** remake this program with `scipy.signal.convolve2d` which is a non-fast function. See how FFT is important!  

Let use a very simple optimization algo to precise our maximum.

***To you:*** Explain the principle of this algo.


In [ ]:
def find_min_local(loss_function, step, dep, nb_iter):

    place=dep
    val=loss_function(place)
    val_plus = loss_function((place + step) % 360)
    val_moins = loss_function((place - step) % 360)

    for i in range(nb_iter):

        if val_plus<val :
            val_moins = val
            val = val_plus
            place = place + step
            val_plus = loss_function((place + step) % 360)

        elif val_moins<val:
            val_plus=val
            val=val_moins
            place = place - step
            val_moins=loss_function((place - step) % 360)

        else:
            step/=2
            val_plus = loss_function((place + step) % 360)
            val_moins = loss_function((place - step) % 360)

        print("iteration %d, place %.5f, value %.10f"%(i,place,val))



    return place, val

In [ ]:
"""Test with a very simple loss function"""
def loss(x):
    return (x/360)**2

arg_mini,mini=find_min_local(loss,122,245,10)
print("final:",arg_mini,mini)

***To you:*** Make a graphical illustration of this minimisation.

In [ ]:
"""Come back to our optimal angle of rotation"""
img1, img2 = get_turned_images(0)

def loss(angle):
    rot = ndimage.rotate(img1, angle, order=2)
    cor = scipy.signal.convolve(rot, img2[::-1,::-1])
    return  -np.max(cor)

arg_mini, mini = find_min_local(loss, 30, 10, 20)

print("final:", arg_mini, mini)

### Rotation and dilatation

***A vous:*** Now you have to match the king, with the head of the king which have been rotated and zoomed.


In [ ]:
img1,img2=get_turned_images(1)

fig,(ax0,ax1)=plt.subplots(1,2)
ax0.imshow(img1)
ax1.imshow(img2);

You can try all the zooms and rotations but it's a little slow. I advise you to do your own little optimization algo (take inspiration from mine). Even if your method is trapped by some local minimums, no problem: try it several times with several different starting points. Remember to draw your loss function in color level, this will allow you to estimate its irregularity. Illustrate graphically the progress of your algorithm (by superimposing a scater plot and the drawing of the loss function). This will allow you to improve it.

You can also use the methods of:

    from scipy.optimize import minimize
    
But they are difficult to parameterize.


Many optimization methods require that you explicitly give them a gradient or a Hessian matrix. In fact, for our simple geometrical transformations, gradient and hessian can be computed explicitly. This gives the Lucas-Kanade methods which belongs to the domain of "non-rigid image matching".

 Such algorithm are include in modern photophones: when you take one photo, it take many different ones, with different parameters (like aperture, exposure), and match them. And also, when you take a panoramic photo, your smarphone must match several images.

## Un exo en 2D

Lissez trouver le bug dans le code suivant

In [ ]:
N=2

In [ ]:
A=np.random.randint(0,10,size=[N,N])
A

In [ ]:
M=np.zeros([N,N])
M[0,0]=1
M

On veut faire la convolution entre A et M. D'abord avec des méthodes toutes simples

In [ ]:
AM=sg.convolve2d(A,M,mode="full")[:N,:N]
AM

In [ ]:
AM=sg.fftconvolve(A,M,mode="full")[:N,:N]
AM

Remarque: on aurait pu faire le padding nous même pour avoir la vrai convolution, mais on a préféré utiliser le mode `"full"`

In [ ]:
A_pad=np.pad(A,[(0,N),(0,N)])
A_pad

In [ ]:
M_pad=np.pad(M,[(0,N),(0,N)])
M_pad

In [ ]:
AM_pad=np.fft.ifft2(np.fft.fft2(A_pad)*np.fft.fft2(M_pad))
AM_pad.real

In [ ]:
AM_pad=np.fft.fft2(np.fft.ifft2(A_pad)*np.fft.ifft2(M_pad))
AM_pad.real*N**2

Mince, on aurait du retomber sur le même résultat.